In [9]:
"""
Deprecated due to generating algorithm review result 
in generate_masks script, and the use of mask editing 
tool in GUi.
"""

%load_ext autoreload
%autoreload 2

import os
import time
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import contours_to_mask
from registration_utilities import find_contour_points

import numpy as np

sys.path.append('/oasis/projects/nsf/csd181/yuncong/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.filters.rank import entropy
from skimage.morphology import remove_small_objects, disk, remove_small_holes, binary_dilation, disk, binary_closing
from skimage.measure import label, regionprops
from skimage.color import rgb2gray
from skimage.io import imread, imsave
from skimage.segmentation import active_contour
from skimage.filters import gaussian, threshold_adaptive
from skimage.util import img_as_ubyte

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from metadata import *
from data_manager import *

In [11]:
stack = 'MD652'

In [12]:
submask_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_submasks' % {'stack': stack})
# gallery_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_submaskGallery' % {'stack': stack})

In [ ]:
# new

In [ ]:
def consolidate_alg_submask_review(alg_decisions):
    """
    Return {submask_ind: bool}
    """

    consolidated_decisions = {}

    if len(alg_decisions) == 0:
        return {}
    else:
        alg_positives = [index for index, l in alg_decisions.iteritems() if l == 1]
        if len(alg_positives) > 0:
            assert len(alg_positives) == 1
            correct_index = alg_positives[0]
        else:
            correct_index = 1

        consolidated_decisions[correct_index] = 1
        for idx in alg_decisions:
            if idx != correct_index:
                consolidated_decisions[idx] = -1
        return {i: r == 1 for i, r in consolidated_decisions.iteritems()}

In [ ]:
n = len(final_masks)
decisions = [0 for _ in range(n)]
undecided = []
negatives = []
positives = []

if n == 1: # If only one mask, assign it positive.
    decisions[0] = 1
    undecided.append(0)
else:
    for i in range(n):
        if np.count_nonzero(final_masks[i]) < 10000:
            decisions[i] = -1
            negatives.append(i)
        else:
            decisions[i] = 0
            positives.append(i)

# If only one undecided and the rest are negative, make the undecided positive.
if len(undecided) == 1 and len(negatives) == n - len(undecided):
    decisions[undecided[0]] = 1

# If there are more than one positives, make all other undecided negative.
if len(positives) >= 1:
    for u in undecided:
        decisions[u] = -1

decisions = consolidate_alg_submask_review(dict(zip(range(1, 1+len(decisions)), decisions)))



In [ ]:
# old

In [18]:
from collections import defaultdict

labels = defaultdict(dict)
for fn in os.listdir(submask_dir):

    mask_fns = os.listdir(submask_dir + '/' + fn)
    masks = [imread(submask_dir + '/' + fn + '/' + mask_fn).astype(bool) for mask_fn in mask_fns]
    
    areas = [np.count_nonzero(mask) for mask in masks]
    indices = [int(mask_fn[:-4].split('_')[-1]) for mask_fn in mask_fns]
    
#     print fn, mask_fns
    
    n = len(indices)
    if n == 1:
        labels[fn][indices[0]] = 1
    else:
        for i in range(n):
            if areas[i] < 10000:
                labels[fn][indices[i]] = -1
            else:
                labels[fn][indices[i]] = 0
    
    undecided = [i for i, l in labels[fn].iteritems() if l == 0]
    negatives = [i for i, l in labels[fn].iteritems() if l == -1]
    positives = [i for i, l in labels[fn].iteritems() if l == 1]
    
    if len(undecided) == 1 and len(negatives) == n - len(undecided):
        labels[fn][undecided[0]] = 1
    
    if len(positives) >= 1:
        for u in undecided:
            labels[fn][u] = -1
            
#     with open('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_submasks/%(fn)s/%(fn)s_masks_algReview.txt' % dict(fn=fn, stack=stack), 'w') as f:
#         for i, lbl in labels[fn].iteritems():
#             f.write('%d %d\n' % (i, lbl))
                
labels.default_factory = None

In [14]:
# add extra cells for correction result
D = {fn: dict(sum([[(str(i)+'_alg', l), (str(i)+'_human', np.nan)] for i, l in d.iteritems()], [])) 
     for fn, d in labels.iteritems()}

In [15]:
sorted_fns = sorted(labels.keys(), key=lambda fn: int(fn.split('-')[1][1:]))

In [16]:
import pandas
df = pandas.DataFrame(data=D, columns=sorted_fns).T
df.to_csv('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_check.csv' % {'stack': stack})

# ... then user reviews this on local machine and prepares the reviewed version.

In [80]:
df_rev = pandas.read_csv('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_check_reviewed.csv' % {'stack': stack},
                        index_col=0)

In [20]:
human_labels_reviewed = {}

for fn, row in df_rev.iterrows():
    human_labels = {int(name[0]): l for name, l in row.dropna().iteritems() if name.endswith('human')}
    alg_labels = {int(name[0]): l for name, l in row.dropna().iteritems() if name.endswith('alg')}        
    
    human_positives = [index for index, l in human_labels.iteritems() if l == 1]
    
    if len(human_positives) == 0:
        # If no human label is given, default to algorithm result; 
        # if algo. is unsure, treat the first mask as correct.
        
        alg_positives = [index for index, l in alg_labels.iteritems() if l == 1]
        if len(alg_positives) > 0:
            assert len(alg_positives) == 1
            correct_index = alg_positives[0]
        else:
            correct_index = 1
    else:
        # if human label is given, use that.
        assert len(human_positives) == 1
        correct_index = human_positives[0]
    
    human_labels[correct_index] = 1
    for idx in alg_labels:
        if idx != correct_index:
            human_labels[idx] = -1

    human_labels_reviewed[fn] = human_labels

In [120]:
# reviewed_mask_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_unsorted_reviewed' % {'stack': stack})
reviewed_mask_dir = create_if_not_exists('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_mask_unsorted' % {'stack': stack})

In [121]:
for fn, d in human_labels_reviewed.iteritems():
    correct_indices = [index for index, l in d.iteritems() if l == 1]
    assert len(correct_indices) == 1
    correct_index = correct_indices[0]
    
    from_fn = os.path.join(submask_dir, '%(fn)s/%(fn)s_submasks_%(corr_idx)d.png' % dict(fn=fn, corr_idx=correct_index))
    to_fn = os.path.join(reviewed_mask_dir, '%(fn)s_mask.png' % dict(fn=fn, corr_idx=correct_index))
    
    execute_command('cp %(from)s %(to)s' % {'from': from_fn, 'to': to_fn})

cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F16-2016.05.18-19.01.39_MD635_1_0046/MD635-F16-2016.05.18-19.01.39_MD635_1_0046_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F16-2016.05.18-19.01.39_MD635_1_0046_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F13-2016.05.18-18.22.59_MD635_2_0038/MD635-F13-2016.05.18-18.22.59_MD635_2_0038_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F13-2016.05.18-18.22.59_MD635_2_0038_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F40-2016.05.19-01.08.51_MD635_2_0119/MD635-F40-2016.05.19-01.08.51_MD635_2_0119_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F40-2016.05.19-01.08.51_MD635_2_0119_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F48-2016.05.19-03.40.12_MD635_2_0143/MD635-F48-2016.05.19-03.40.12_MD635_2_0143_submasks_1.png /home/yuncong/CSHL_data_proc

Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0
Child returned 0


cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F163-2016.05.20-10.17.45_MD635_3_0489/MD635-F163-2016.05.20-10.17.45_MD635_3_0489_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F163-2016.05.20-10.17.45_MD635_3_0489_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F62-2016.05.19-08.18.25_MD635_2_0185/MD635-F62-2016.05.19-08.18.25_MD635_2_0185_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F62-2016.05.19-08.18.25_MD635_2_0185_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F157-2016.05.20-13.32.17_MD635_1_0469/MD635-F157-2016.05.20-13.32.17_MD635_1_0469_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F157-2016.05.20-13.32.17_MD635_1_0469_mask.png


Child returned 0
Child returned 0
Child returned 0


cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F132-2016.05.20-04.14.48_MD635_1_0394/MD635-F132-2016.05.20-04.14.48_MD635_1_0394_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F132-2016.05.20-04.14.48_MD635_1_0394_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F11-2016.05.18-17.57.22_MD635_1_0031/MD635-F11-2016.05.18-17.57.22_MD635_1_0031_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F11-2016.05.18-17.57.22_MD635_1_0031_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F23-2016.05.18-20.33.07_MD635_1_0067/MD635-F23-2016.05.18-20.33.07_MD635_1_0067_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F23-2016.05.18-20.33.07_MD635_1_0067_mask.png


...

Child returned 0
Child returned 0
Child returned 0


cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F118-2016.05.20-01.05.52_MD635_2_0353/MD635-F118-2016.05.20-01.05.52_MD635_2_0353_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F118-2016.05.20-01.05.52_MD635_2_0353_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F4-2016.05.18-16.52.52_MD635_3_0012/MD635-F4-2016.05.18-16.52.52_MD635_3_0012_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F4-2016.05.18-16.52.52_MD635_3_0012_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F155-2016.05.20-09.19.32_MD635_3_0465/MD635-F155-2016.05.20-09.19.32_MD635_3_0465_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F155-2016.05.20-09.19.32_MD635_3_0465_mask.png


Child returned 0
Child returned 0
Child returned 0
Child returned 0


cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F10-2016.05.18-17.44.17_MD635_1_0028/MD635-F10-2016.05.18-17.44.17_MD635_1_0028_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F10-2016.05.18-17.44.17_MD635_1_0028_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F85-2016.05.19-15.36.47_MD635_2_0254/MD635-F85-2016.05.19-15.36.47_MD635_2_0254_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F85-2016.05.19-15.36.47_MD635_2_0254_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F120-2016.05.20-01.39.14_MD635_1_0358/MD635-F120-2016.05.20-01.39.14_MD635_1_0358_submasks_1.png /home/yuncong/CSHL_data_processed/MD635/MD635_mask_unsorted/MD635-F120-2016.05.20-01.39.14_MD635_1_0358_mask.png
cp /home/yuncong/CSHL_data_processed/MD635/MD635_submasks/MD635-F6-2016.05.18-17.10.18_MD635_1_0016/MD635-F6-2016.05.18-17.10.18_MD635_1_0016_submasks_1.png /home/yuncong/CSHL_data_pro

Child returned 0


# Learn

In [41]:
from scipy.stats import entropy

labels = defaultdict(dict)
for fn in os.listdir(submask_dir):
    img_rgb = imread('/home/yuncong/CSHL_data/%(stack)s/%(fn)s.tif' % dict(stack=stack, fn=fn))

    mask_fns = os.listdir(submask_dir + '/' + fn)
    masks = [imread(submask_dir + '/' + fn + '/' + mask_fn).astype(bool) for mask_fn in mask_fns]
    vals = [img_rgb[mask, 2] for mask in masks]
    ents = [entropy(v) for v in vals]
    stds = [np.std(v) for v in vals]
    
    vals_r = [img_rgb[mask, 0] for mask in masks]
    ents_r = [entropy(v) for v in vals_r]
    stds_r = [np.std(v) for v in vals_r]
    
    vals_g = [img_rgb[mask, 1] for mask in masks]
    ents_g = [entropy(v) for v in vals_g]
    stds_g = [np.std(v) for v in vals_g]
    
    areas = [np.count_nonzero(mask) for mask in masks]
    indices = [int(mask_fn[:-4].split('_')[-1]) for mask_fn in mask_fns]
    n = len(indices)
    if n == 1:
        labels[fn][indices[0]] = (1, ents[0], stds[0], ents_r[0], stds_r[0], ents_g[0], stds_g[0])
    else:
        for i in range(n):
            if areas[i] < 10000:
                labels[fn][indices[i]] = (-1, ents[i], stds[i], ents_r[i], stds_r[i], ents_g[i], stds_g[i])
                
labels.default_factory = None

In [64]:
D = np.array([dd for d in labels.itervalues() for dd in d.itervalues()])
Y = D[:,0]
X = D[:, 1:3]

In [65]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=.1)
clf.fit(X, Y)
print clf.coef_, clf.intercept_

n = len(D)
print 'Train acc:', np.count_nonzero((clf.predict_proba(X)[:, 1] > 0.5) == (Y == 1)) / float(n)

[[ 0.41265194 -0.00855474]] [-1.99669771]
Train acc: 0.842519685039


In [66]:
predictions = defaultdict(dict)

for fn in os.listdir(submask_dir):
    img_rgb = imread('/home/yuncong/CSHL_data/%(stack)s/%(fn)s.tif' % dict(stack=stack, fn=fn))

    mask_fns = os.listdir(submask_dir + '/' + fn)
    masks = [imread(submask_dir + '/' + fn + '/' + mask_fn).astype(bool) for mask_fn in mask_fns]
    
    vals = [img_rgb[mask, 2] for mask in masks]
    ents = [entropy(v) for v in vals]
    stds = [np.std(v) for v in vals]
    
    vals_r = [img_rgb[mask, 0] for mask in masks]
    ents_r = [entropy(v) for v in vals_r]
    stds_r = [np.std(v) for v in vals_r]
    
    vals_g = [img_rgb[mask, 1] for mask in masks]
    ents_g = [entropy(v) for v in vals_g]
    stds_g = [np.std(v) for v in vals_g]
    
    areas = np.array([np.count_nonzero(mask) for mask in masks])
    indices = [int(mask_fn[:-4].split('_')[-1]) for mask_fn in mask_fns]
#     scores = clf.predict_proba(np.c_[ents, stds, ents_r, stds_r, ents_g, stds_g])[:,1]
    scores = clf.predict_proba(np.c_[ents, stds])[:,1]
#     y = scores > 0
    n = len(indices)
    for i in range(n):
        predictions[fn][indices[i]] = (scores[i], ents[i], stds[i], ents_r[i], stds_r[i], ents_g[i], stds_g[i])
                
predictions.default_factory = None

In [67]:
np.count_nonzero([np.count_nonzero([dd[0] > .5 for dd in d.itervalues()]) == 1 
                  for fn, d in sorted(predictions.items())]) / float(len(predictions))

0.671264367816092

In [63]:
np.count_nonzero([np.count_nonzero([dd[0] > .5 for dd in d.itervalues()]) == 1 
                  for fn, d in sorted(predictions.items())]) / float(len(predictions))

0.6735632183908046

In [30]:
for fn, d in sorted(predictions.items()):
    print fn
    print d

MD635-F1-2016.05.18-16.23.55_MD635_1_0001
{1: (0.57592460634799425, 9.4961567909212441, 188.88298762329401), 2: (0.019003734200276481, 4.2162216573936142, 430.99717060169513)}
MD635-F1-2016.05.18-16.23.55_MD635_2_0002
{1: (0.54674999804326818, 9.3757649037961741, 196.92972909976601)}
MD635-F1-2016.05.18-16.23.55_MD635_3_0003
{1: (0.7024007975007881, 9.6317876217615392, 130.81893293147476)}
MD635-F10-2016.05.18-17.44.17_MD635_1_0028
{1: (0.76095416966302576, 10.69381201555241, 147.07719304975049)}
MD635-F10-2016.05.18-17.44.17_MD635_2_0029
{1: (0.76225978499679681, 10.729165349722916, 147.94192833193506)}
MD635-F10-2016.05.18-17.44.17_MD635_3_0030
{1: (0.75230140300595871, 10.733340492487867, 154.47718309712192), 2: (0.12723172432884941, 5.2233029476985955, 243.65078852202561)}
MD635-F100-2016.05.19-20.25.22_MD635_1_0298
{1: (0.90747894938519436, 12.444578854364272, 99.985272046396119)}
MD635-F100-2016.05.19-20.25.22_MD635_2_0299
{1: (0.8838738248736433, 12.425716358184289, 128.71947207